In [4]:
import requests
from bs4 import BeautifulSoup
import lxml.html
from selenium import webdriver
import time
import pandas as pd
import re
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Func to login, and scrape mnf, mnf_link base on BC location

In [5]:
def scrape_more_beer(pages):
    driver = webdriver.Chrome('/home/sam/Downloads/chromedriver')
    url = 'https://www.beeradvocate.com/community/login/'
    driver.get(url)
    
    loginelement = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//form[@class="xenForm formOverlay"]//dd//input[@name ="login"]')))
    loginelement.send_keys('suzie102')

    pwelement = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//form[@class="xenForm formOverlay"]//dl[@class ="ctrlUnit"]//dd//ul//li[@id = "ctrl_pageLogin_registered_Disabler"]//input[@name ="password"]')))
    pwelement.send_keys('Beer@addiction102')

    page_click = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//form[@class="xenForm formOverlay"]//dl[@class ="ctrlUnit submitUnit"]//dd//input[@type ="submit"]')))
    page_click.click()
    
    mnf = []
    mnf_link = []
    for page_num in tqdm(range(0,pages,20)):
        new_url = "https://www.beeradvocate.com/place/list/?start=%d&c_id=CA&s_id=BC&brewery=Y&sort=name" %(page_num)
        req = driver.get(new_url)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        page_source = driver.page_source
        soup = lxml.html.fromstring(page_source)
        mnf_i = soup.xpath('//b/text()')[4::5]
        mnf_link_i = soup.xpath('//div[@id ="ba-content"]/table/tbody/tr/td/a/@href')[1::4]
        #print(mnf_link_i)
        mnf.append(mnf_i)
        mnf_link.append(mnf_link_i)
    mnf_link_rm_ns_ls = [ i for m in mnf_link for i in m[1:]]
    mnf_rm_ns_ls = [ i for m in mnf for i in m[:-1]]
    mnf = pd.DataFrame({'mnf': mnf_rm_ns_ls, "mnf_link": mnf_link_rm_ns_ls})
    return mnf

In [6]:
mnf = scrape_more_beer(195)

100%|██████████| 10/10 [00:37<00:00,  3.77s/it]


# func to get product link from mnf 

In [11]:
def product_name(links):
    driver = webdriver.Chrome('/home/sam/Downloads/chromedriver')
    url = 'https://www.beeradvocate.com/community/login/'
    driver.get(url)
    
    loginelement = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//form[@class="xenForm formOverlay"]//dd//input[@name ="login"]')))
    loginelement.send_keys('suzie102')

    pwelement = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//form[@class="xenForm formOverlay"]//dl[@class ="ctrlUnit"]//dd//ul//li[@id = "ctrl_pageLogin_registered_Disabler"]//input[@name ="password"]')))
    pwelement.send_keys('Beer@addiction102')

    page_click = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//form[@class="xenForm formOverlay"]//dl[@class ="ctrlUnit submitUnit"]//dd//input[@type ="submit"]')))
    page_click.click()
    
    product = []
    pro_link =[]
    
    for link_i in tqdm(links):
        new_url = "https://www.beeradvocate.com"+link_i
        req = driver.get(new_url)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        page_source = driver.page_source
        soup = lxml.html.fromstring(page_source)
        product_i = soup.xpath('//b/text()')
        pro_link_i = soup.xpath('//div[@style ="font-size:1em;"]/table/tbody/tr/td/a/@href')[0::2]
     
        product.append(product_i)
        pro_link.append(pro_link_i)
    clean = [ i for m in product for i in m [ m.index(' | ')+2 :-1:3 ] ]
    product_link = [ i for m in pro_link for i in m  ]
    assert(len(clean )==len(product_link))
    product_list = pd.DataFrame({"product":clean, "pro_link":product_link})
    return product_list

In [12]:
product_list = product_name(mnf.mnf_link)

100%|██████████| 195/195 [12:36<00:00,  3.88s/it]


# func to get attributes of products

In [14]:
#'rate': 'detail_rating', 'avg': 'avg_rating','number_of_rev':'number_of_reviews', 'avg_user_rev': 'avg_per_user', 'reviews': 'text_reviews',

In [15]:
def webstite_scrape_p2 (beer_link):

    driver = webdriver.Chrome('/home/sam/Downloads/chromedriver')
    url = 'https://www.beeradvocate.com/community/login/'
    driver.get(url)
    
    loginelement = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//form[@class="xenForm formOverlay"]//dd//input[@name ="login"]')))
    loginelement.send_keys('suzie102')

    pwelement = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//form[@class="xenForm formOverlay"]//dl[@class ="ctrlUnit"]//dd//ul//li[@id = "ctrl_pageLogin_registered_Disabler"]//input[@name ="password"]')))
    pwelement.send_keys('Beer@addiction102')

    page_click = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//form[@class="xenForm formOverlay"]//dl[@class ="ctrlUnit submitUnit"]//dd//input[@type ="submit"]')))
    page_click.click()
    
    mnf =[]
    style =[]
    alc =[]
    ratings = [] #number of ratings
    avg_rating = [] 
    number_of_reviews = []
    
    for link in tqdm(beer_link): 
        new_url = "https://www.beeradvocate.com/"+link
        print(new_url)
        driver.get(new_url)

        time.sleep(5)
        page_source = driver.page_source
        soup = lxml.html.fromstring(page_source)
        
        mnf_i = soup.xpath('//dd[@class = "beerstats"]/a[@class = "Tooltip"]/text()')
        style_i = soup.xpath('//dd[@class = "beerstats"]/a[@class = "Tooltip"]/b/text()')
        alc_i = soup.xpath('//dd[@class = "beerstats"]/span[@class = "Tooltip"]/b/text()')
        ratings_i= soup.xpath('//dd[@class = "beerstats"]/span[@class = "ba-ratings Tooltip"]/text()')
        
        avg_i = soup.xpath('//dd[@class = "beerstats"]/b/span[@class = "ba-ravg Tooltip"]/text()')
        number_of_reviews_i = soup.xpath('//dd[@class = "beerstats"]/span[@class = "ba-reviews Tooltip"]/text()')
        
        mnf.append( mnf_i)
        style.append(style_i)
        alc.append(alc_i)
        ratings.append(ratings_i)
        avg_rating.append(avg_i)
        number_of_reviews.append(number_of_reviews_i)
    beer_details = pd.DataFrame(list(zip(mnf,style, alc, ratings, avg, number_of_reviews)), columns =["mnf","style", "alc", "number_of_ratings", "avg_ratings", "number_of_reviews"])
    for i in beer_details.iloc[:,1:]:
        print(i)
        beer_details[i] = beer_details[i].apply(lambda x:x[0] if len(x)>0 else "no info")
    beer_details.mnf =beer_details.mnf.apply(lambda x: x[-1] if len(x)>0 else x)
    return 

In [ ]:
beer_details= webstite_scrape_p2(product_list.pro_link)

  0%|          | 0/3354 [00:00<?, ?it/s]

https://www.beeradvocate.com//beer/profile/32043/238773/


  0%|          | 1/3354 [00:06<5:52:57,  6.32s/it]

https://www.beeradvocate.com//beer/profile/32043/185753/


  0%|          | 2/3354 [00:12<5:46:21,  6.20s/it]

...

https://www.beeradvocate.com//beer/profile/13236/86623/


  9%|▉         | 300/3354 [32:10<5:05:35,  6.00s/it]

https://www.beeradvocate.com//beer/profile/13236/189379/


  9%|▉         | 301/3354 [32:16<5:00:59,  5.92s/it]

https://www.beeradvocate.com//beer/profile/13236/38074/


In [ ]:
beer_details['name'] = product_list['product']
beer_details['rev_link'] =product_list['pro_link']

# Scrape reviews

In [ ]:
def webstite_scrape_p3 (beer_link, number_of_reviews):

    driver = webdriver.Chrome('/home/sam/Downloads/chromedriver')
    url = 'https://www.beeradvocate.com/community/login/'
    driver.get(url)
    
    login_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//form[@class="xenForm formOverlay"]//dd//input[@name ="login"]')))
    login_element.send_keys('suzie102')

    pw_element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//form[@class="xenForm formOverlay"]//dl[@class ="ctrlUnit"]//dd//ul//li[@id = "ctrl_pageLogin_registered_Disabler"]//input[@name ="password"]')))
    pw_element.send_keys('Beer@addiction102')

    page_click = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//form[@class="xenForm formOverlay"]//dl[@class ="ctrlUnit submitUnit"]//dd//input[@type ="submit"]')))
    page_click.click()

    rate = []
    reviews =[]
    avg_user =[]
    for link, i in tqdm(zip(beer_link, number_of_reviews)): 
        for pages_i in range(0,int(i)+1,25): #site shows 25 resulst/page)        
            new_url = "https://www.beeradvocate.com"+link+'?view=beer&sort=&start=%d' %(pages_i)
            driver.get(new_url)
            #print(driver.find_element_by_name("hideRatings").is_selected())
            #check_box = WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.XPATH, '//form[@style="display:inline;margin:0;padding:0;"]//input[@type = "checkbox"]')))#check_box.click()
            #check_box.click()
            time.sleep(5)
            page_source = driver.page_source
            soup = BeautifulSoup(page_source,'html.parser')
        
            rate_i = [ i.get_text() for i in soup.find_all('span', class_ = "muted")][8::3]
            rate.append(rate_i)
            
            reviews_i = [ i.get_text() for i in soup.find_all('div')]
            reviews.append(reviews_i)
            
            avg_i = [i.get_text() for i in soup.find_all('span', class_= "BAscore_norm")]
            print(new_url)
            print(avg_i)
            avg_user.append(avg_i)
    return rate, reviews, avg_user

In [ ]:
detail_rating, text_reviews, avg_per_user = webstite_scrape_p3 (beer_details.rev_link, beer_details.number_of_reviews

In [ ]:
name=[]
mnf =[]

for mnf_i,beer, link, i in zip(beer_details.mnf,beer_details.name, beer_details.rev_link, beer_details.number_of_reviews): 
    for pages_i in range(0,int(i)+1,25): #site shows 25 resulst/page)
        new_url = link+'?view=beer&sort=&start=%d'%(pages_i)
        name.append(beer)
        mnf.append(mnf_i)
print(len(name))

In [ ]:
temp =pd.DataFrame(list(zip(mnf,name,rate, reviews, avg_user)), columns =['mnf','name','detail_ratings', 'reviews', 'avg_per_user'])

# cleaning reviews 

In [ ]:
def text_cleaning (beerreview):
    text =[]
    for review in tqdm(beerreview):   
        clean_rev = [i.replace("\n","") for i in review[0].split("\xa0") if len(i)>0]
        clean_rev2 =[ re.findall(r'rDev(.*)',i) for i in clean_rev if len(re.findall(r'rDev(.*)',i))>0]
        text.append(clean_rev2)
    return text

In [ ]:
def final_clean (text):
    patt="overall:\s*\d\.?\d?\d?"
    clean = [re.findall(patt+"(.*)",i)[0] for i in text if len(re.findall(patt+"(.*)",i))>0]
    return clean

In [ ]:
temp['clean_text'] = text_cleaning(reviews)
temp['rm_nested_ls'] = temp['clean_text'].apply(lambda x: [i for m in x for i in m])
temp['text_reviews'] = temp['rm_nested_ls'].apply(final_clean)

In [ ]:
temp.drop(['reviews','clean_text','rm_nested_ls'], axis = 1, inplace = True)

# Merge beer attribute and reviews

In [ ]:
temp['comb'] = temp.mnf+temp.name
temp = temp.groupby('comb').sum()
temp.reset_index(inplace = True)

In [ ]:
beer_details['comb']=beer_details.mnf+beer_details.name

In [ ]:
additonal_reviews = beer_details.merge(temp, on = 'comb')